In [4]:
import cv2
import ntpath
import random
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from sklearn.utils import shuffle
from tensorflow.keras import layers
from keras.models import load_model
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from imgaug import augmenters  as iaa
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, LSTM, Reshape, Input, Concatenate, ZeroPadding2D, GlobalMaxPool2D, BatchNormalization, Normalization

挂载谷歌硬盘，读取并导入数据集

In [5]:

pickle_file = open('E:/training_data_1.pkl', 'rb')
training_data_1 = np.asarray(pickle.load(pickle_file))
pickle_file = open('E:/training_data_2.pkl', 'rb')
training_data_2 = np.asarray(pickle.load(pickle_file))
pickle_file = open('E:/training_data_3.pkl', 'rb')
training_data_3 = np.asarray(pickle.load(pickle_file))
pickle_file = open('E:/training_data_4.pkl', 'rb')
training_data_4 = np.asarray(pickle.load(pickle_file))
pickle_file = open('E:/training_data_5.pkl', 'rb')
training_data_5 = np.asarray(pickle.load(pickle_file))
pickle_file = open('E:/training_data_6.pkl', 'rb')
training_data_6 = np.asarray(pickle.load(pickle_file))
pickle_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'E:/training_data_5.pkl'

In [ ]:
training_data = np.row_stack((training_data_1, training_data_2, training_data_3, training_data_4, training_data_5, training_data_6))
del training_data_1, training_data_2, training_data_3, training_data_4, training_data_5, training_data_6 # clean up memory

training_data.shape

In [ ]:
for data in training_data:
  data[0] = cv2.cvtColor(data[0], cv2.COLOR_BGR2GRAY)

从数据集中随机抽取一张图并查看相应的键盘输入。可以看到，输入由油门和转向两部分组成

In [ ]:
def show_img(index):
  plt.title('Sample Image, key = '+ str(training_data[index][1]))
  plt.imshow(training_data[index][0])
  plt.show()

index = np.random.randint(0,len(training_data))
show_img(index)

分别绘制油门和转向的柱状图。可以看到，油门的值聚集在1，转向的值聚集在0。这是符合常理的，因为开车时大部分时间都在踩油门沿道路直线行驶。不过，如果不处理数据直接训练，训练出的模型会在绝大部分时候选择踩油门直行。因此，必须删除部分踩油门直行的数据，防止造成偏差。

In [ ]:
throttle = [a[1][0] for a in training_data]
steering = [a[1][1] for a in training_data]

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].hist(throttle,3)
axs[0].set_title('Throttle')
axs[1].hist(steering,3)
axs[1].set_title('Steering')

In [ ]:
drop_percent = 0.8

new = []
for data in training_data:
  if data[1][0]==1 and data[1][1]==0:
    if np.random.randint(1,11)/10>=drop_percent:
      new.append(data)
  else:
    new.append(data)


throttle = [a[1][0] for a in new]
steering = [a[1][1] for a in new]

training_data = np.asarray(new)
del new

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].hist(throttle,3)
axs[0].set_title('Throttle')
axs[1].hist(steering,3)
axs[1].set_title('Steering')

del throttle, steering, pickle_file

除了删除部分踩油门直行的数据，还可以通过镜像图片，调整亮度等方式人为地多制造一些转弯数据。

In [ ]:
def img_flip(input, percent):
  output = input  # make a copy
  for data in input:
    img, key = data
    if key[1]!=0 and np.random.randint(1,11)/10<=percent:   # if has steerling control
      img_flipped = cv2.flip(img,1)  # flip the img
      key_reversed = [key[0], -key[1]] # reverse the steering input
      output = np.row_stack((output,[img_flipped,key_reversed]))

  return output

def img__brightness(input, percent):
  output = input  # make a copy
  for data in input:
    if np.random.randint(1,11)/10<=percent:
      img, key = data
      brightness = iaa.Multiply((0.7, 1.5))
      img = brightness.augment_image(img)
      output = np.row_stack((output,[img,key]))

  return output

img = training_data[np.random.randint(0,len(training_data))][0]
img_1 = cv2.flip(img,1)
img_2 = iaa.Multiply((0.2, 1.2)).augment_image(img)

fig, axs = plt.subplots(1, 3, figsize=(20, 5))
axs[0].imshow(img)
axs[0].set_title('Original')
axs[1].imshow(img_1)
axs[1].set_title('Flipped')
axs[2].imshow(img_2)
axs[2].set_title('Brightness')


training_data = img_flip(training_data,0.9)
training_data = img__brightness(training_data,0.3)

In [ ]:
def key_to_class(key_data):
  class_list = np.asarray([[0,1],[0,-1],[0,0],[1,1],[1,-1],[1,0],[-1,1],[-1,-1],[-1,0]])

  out = []
  for data in key_data:
    out.append(np.where(np.all(class_list==data,axis=1)))
  
  return np.asarray(out).reshape(key_data.shape[0],1)


X = [i[0] for i in training_data]
t = [i[1] for i in training_data]


t = key_to_class(np.array(t))


X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2)

del X,t

X_test = np.array(X_test)
X_train = np.array(X_train)


In [ ]:
del training_data

In [ ]:
model = Sequential([

  layers.Rescaling(1./255, input_shape=(150, 350, 1)),


  layers.Conv2D(24,kernel_size=(3,3), strides=(2,2),padding='same',activation='relu'),


  layers.Conv2D(48,kernel_size=(5,5), strides=(2,2),padding='same',activation='relu'),

  

  layers.Conv2D(64,kernel_size=(3,3), strides=(2,2),padding='same',activation='relu'),

  layers.MaxPooling2D(pool_size=(2,2)),

  layers.Normalization(),
  #1st Dense Layer
  layers.Flatten(),

  #2nd Dense Layer
  layers.Dense(100, activation='relu'),#500
  layers.Dropout(0.3),

  layers.Dense(64, activation='relu'),
  layers.Dropout(0.3),
  #output
  layers.Dense(9, activation='softmax')


  #layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.0001)),

])
  
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

print (model.summary())

In [ ]:
epochs=100
early_stopping_callback = EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=5, verbose=1)

history = model.fit(X_train,t_train,            
  batch_size=128,epochs=epochs, callbacks=early_stopping_callback,
  validation_data=(X_test,t_test),validation_freq=1
)

In [ ]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(1,2,1)
plt.plot(acc,label='Training acc')
plt.plot(val_acc,label='val acc')
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss,label='Training loss')
plt.plot(val_loss,label='val loss')
plt.legend()
plt.show()

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/ETS2AD/model_6.h5')

In [ ]:
a=[]
for i in range(30):

  a.append(np.argmax(model.predict(X_test[i].reshape((-1,150, 350)))))
  
print(a)
print(t_test[0:30])